In [1]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

In [2]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

### Writing PyG Dataset

In [3]:
import torch

In [4]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [5]:
from torch_geometric.utils import from_networkx, to_networkx

In [6]:
def clustering_coefficient(G, node):
    ns = [n for n in G.neighbors(node)]
    if len(ns) <= 1:
        return 0
    
    numerator = 0
    denominator = len(ns) * (len(ns) - 1) / 2
    for i in range(0, len(ns)):
        for j in range(i+1, len(ns)):
            n1, n2 = ns[i], ns[j]
            numerator += G.has_edge(n1, n2)
    
    return numerator / denominator
                

In [7]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 2)
    ind = 0
    for node in G.nodes():
        # set node degree as feature
        x[ind][0] = G.degree[node]
        x[ind][1] = clustering_coefficient(G, node)
        ind += 1
    return x

In [8]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        for ind, graph in enumerate(total_laman_data[0]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        for ind, graph in enumerate(total_laman_data[1]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [9]:
DATA_PATH = "../data-2d/data/4096-20-4-entries-med.pkl.gz"

In [10]:
laman_data = LamanDataset("", DATA_PATH)

In [11]:
laman_data[0]

Data(edge_index=[2, 122], x=[32, 2], label=[1], num_nodes=32)

## Split into Train / Test

In [12]:
from torch.utils.data import random_split

proportions = [.6, .4]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [16]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 256, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 256, shuffle=True)

In [17]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  10
Number of test batches:  7


In [18]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 19278], x=[5163, 2], label=[256], num_nodes=5163, batch=[5163], ptr=[257])


## Model Architecture & Training

In [19]:
from gin.gin import GIN

In [20]:
model = GIN(num_features=2)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [21]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

# scheduler = ReduceLROnPlateau(optimizer, 'min', min_lr=1e-6, verbose=True, patience=10)

In [22]:
def train(model, data, features_to_use):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, model

In [23]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [49]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, model = train(model, train_loader, [0, 1])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader, [0, 1]), check_accuracy(model, test_loader, [0, 1])
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...
Epoch 0 | Train loss 0.036579158157110214
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 1 | Train loss 0.035865191370248795
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 2 | Train loss 0.03204244375228882
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 3 | Train loss 0.038115695118904114
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 4 | Train loss 0.03280855342745781
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 5 | Train loss 0.032691117376089096
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 6 | Train loss 0.036023594439029694
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 7 | Train loss 0.033724166452884674
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 8 | Train loss 0.03542527183890343
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 9 | Train loss 0.030913038179278374
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 10 | Train loss 0.032389625906944275
Train Accuracy 100.0 | Test Accuracy 100.0
Epoch 11 | Train loss 0.033181332051754


KeyboardInterrupt: 

In [25]:
# sanity check

In [26]:
# test on best model

In [50]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [51]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)

In [52]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [53]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [54]:
# no triangle and rigid
rigid = nx.Graph()
rigid.add_edge(0, 1)
rigid.add_edge(0, 2)
rigid.add_edge(0, 4)
rigid.add_edge(1, 2)
rigid.add_edge(1, 5)
rigid.add_edge(2, 3)
rigid.add_edge(3, 4)
rigid.add_edge(3, 5)
rigid.add_edge(4, 5)

In [55]:
toy_problems = [square, square_bar, triangle, pentagon, rigid]
labels = [1, 0, 0, 1, 0]

for index, toy_problem in enumerate(toy_problems):
    graph_as_data = from_networkx(toy_problem)
    graph_as_data.x = generate_feature_vector(toy_problem)
    graph_as_data.label = labels[index]
    validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
    for batch in validation_set:
        pred = model(batch.x[:, [0, 1]], batch.edge_index, batch.batch)
        print(pred[0])
    

tensor([[0.9987]], grad_fn=<SigmoidBackward0>)
tensor([[0.3450]], grad_fn=<SigmoidBackward0>)
tensor([[0.8439]], grad_fn=<SigmoidBackward0>)
tensor([[1.0000]], grad_fn=<SigmoidBackward0>)
tensor([[0.2499]], grad_fn=<SigmoidBackward0>)


In [41]:
graph_as_data

Data(edge_index=[2, 10], num_nodes=5, x=[5, 2], label=1)

In [42]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [43]:
for batch in validation_set:
    pred = bestModel(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

tensor([[1.]], grad_fn=<SigmoidBackward0>)
tensor([[41.9681]], grad_fn=<AddmmBackward0>)


In [35]:
class LamanTestDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_test.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [36]:
# add functionality to support a test dataset
TEST_DATA_PATH = "../data-2d/data/test-dataset-30loc-5std.pkl.gz"
laman_test_set = LamanTestDataset("", TEST_DATA_PATH)

In [37]:
from torch_geometric.loader import DataLoader
laman_test_loader = DataLoader(laman_test_set, batch_size = 2, shuffle=True)

In [40]:
random_test_acc = check_accuracy(model, laman_test_loader, [0, 1])
print(f"Accuracy {random_test_acc}")

Accuracy 100.0


In [ ]:
# generate statistics on the data

In [48]:
# test the clustering coefficient

In [50]:
clustering_coefficient(square, 0)

0.0

In [51]:
clustering_coefficient(triangle, 0)

1.0

In [52]:
clustering_coefficient(square_bar, 0)

0.6666666666666666

In [54]:
# generate graph correlating clustering coefficient to rigidity

In [59]:
for item in train_data:
    item = to_networkx(item)
    print(type(item))
    
    break

<class 'networkx.classes.digraph.DiGraph'>


In [ ]:
torch_geometric.utils.convert.to_networkx()

# Scratch Work: Sahil

In [60]:
total_laman_data = None
with gzip.open(DATA_PATH, 'r') as f:
    total_laman_data = pickle.load(f)

In [61]:
sample_graph = total_laman_data[0][0]

In [63]:
print(type(sample_graph))

<class 'networkx.classes.graph.Graph'>


In [67]:
to_data = from_networkx(sample_graph)
from_data = to_networkx(to_data, to_undirected = True)

In [68]:
print(type(from_data))

<class 'networkx.classes.graph.Graph'>


In [80]:
def compute_min_clustering_coefficient(G):
    min_coefficient = 1
    for node in G.nodes():
        min_coefficient = min(min_coefficient, clustering_coefficient(G, node))
        
    return min_coefficient

In [82]:
for index, sample_graph in enumerate(train_data):
    label = sample_graph.label
    networkx_sample_graph = to_networkx(sample_graph, to_undirected = True)
    print(label, " ", index, " ", compute_min_clustering_coefficient(networkx_sample_graph))
    
    if index == 10:
        break

tensor([1])   0   0.0
tensor([1])   1   0.0
tensor([0])   2   0.0
tensor([0])   3   0.0
tensor([0])   4   0.0
tensor([0])   5   0.0
tensor([1])   6   0.0
tensor([0])   7   0.0
tensor([0])   8   0.0
tensor([1])   9   0.0
tensor([0])   10   0.0


In [81]:
print(compute_min_clustering_coefficient(from_data))

0.0


In [83]:
compute_min_clustering_coefficient(triangle)

1

In [ ]:
# what if instead of training a gnn – you just trained on 

In [84]:
# train a binary classifier on just the degrees of the nodes



In [85]:
# train a binary classifier on just the triangle feature

In [86]:
# train a binary classifier on the triangle features and the degree

In [87]:
# train a network with just degree of the node 

In [91]:
model_just_degree = GIN(num_features=1)
print(model_just_degree)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=1, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=16, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True,

In [34]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, h = train(train_loader, [0, 1])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model_just_degree, train_loader), check_accuracy(model_just_degree, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...


TypeError: train() missing 1 required positional argument: 'features_to_use'

In [1]:
generate_feature_vector(sample_graph)

NameError: name 'generate_feature_vector' is not defined

In [110]:
sample_graph.x[:, [0, 1]]

tensor([[5.0000, 0.3000],
        [7.0000, 0.1429],
        [3.0000, 0.0000],
        [5.0000, 0.0000],
        [6.0000, 0.1333],
        [3.0000, 0.0000],
        [5.0000, 0.3000],
        [6.0000, 0.1333],
        [3.0000, 0.3333],
        [2.0000, 0.0000],
        [3.0000, 0.3333],
        [5.0000, 0.2000],
        [3.0000, 0.0000],
        [2.0000, 0.0000],
        [2.0000, 1.0000],
        [3.0000, 0.6667],
        [3.0000, 0.3333]])